# TensorFlow Usage

**TensorFlow**는 구글에서 만든 open source software library이다.
Data flow graphs를 이용하여 numerical 계산을 수행할 수 있다.

---
## Installation

#### Basic
* official site (<https://tensorflow.org>)

#### Requirements
* Environments: Ubuntu 16.04
* python 2.7 or 3.6
* python-pip
* python-numpy
* anaconda
* Cuda Toolkit 8.0 (optional)
* cuDNN v6 (optional)

#### Install TensorFlow
```shell
$ sudo pip3 install tensorflow       # Python 3.6; CPU only
$ sudo pip3 install tensorflow-gpu   # Python 3.6; GPU support
```
* more information about installation (<https://www.tensorflow.org/install/>)

---
## 자신의 코드를 작성하는 법

### Flow of general machine learning
* 풀고자 하는 문제가 supervised-learning인지 unsupervised-learning인지 파악한다.
* Supervised-learning일 경우 적절할 데이터 셋을 모아야 한다.
  * input data -- label 이 쌍으로 존재하는 데이터 셋을 찾아서 모아야 한다.
  * 데이터는 많으면 많을수록 좋다
* model을 구현한다.
* Train (and validation)
* Test
* Service (deploy model)

### Machine learning flow를 따라 구현하기

#### 참고할 만한 좋은 reference codes

* cifar10 code (<https://www.tensorflow.org/tutorials/deep_cnn>)
* inception code (<https://github.com/tensorflow/models/tree/master/research/inception>)
* show_and_tell code (<https://github.com/tensorflow/models/tree/master/research/im2txt>)
* other modles (<https://github.com/tensorflow/models/>)
* **official pretrained model** (<https://github.com/tensorflow/models/tree/master/research/slim>)
* **official API docs** (<https://www.tensorflow.org/api_docs/>)

#### Various high-level wrapper

* [`tf.contrib.slim`](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/contrib/slim)
* [`keras`](https://keras.io/)
* ...
* 저는 주로 `tf.contrib.slim`을 씁니다.

---
### 0. Commons

* 크게 파일을 여섯 가지 형태로 구분하고 각각을 구현한다.
  * `train.py`
    * 실제 학습에 필요한 flow를 구현
  * `eval.py`
    * 학습된 모델을 평가하는 flow를 구현
  * `model.py`
    * 만들고자 하는 모델 구현
  * `inference.py`
    * service 환경처럼 개별 데이터 inference하는 코드 구현
  * `image_processiong.py`
    * Data 입출력 및 augmentation에 관련된 코드 구현
  * `configuration.py`
    * Hyper-parameter들 모아놓은 코드
* first import (TensorFlow recommendation)

```python
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
```

* second import (우리가 필요한 library들)

```python
import os
import numpy as np
import time
import matplotlib.pyplot as plt

import tensorflow as tf

slim = tf.contrib.slim  # my recommendation
```

* Set Flags (hyper-parameters) **요즘은 parser로 바뀐 것 같음**

```python
tf.app.flags.DEFINE_string('train_dir',
                           'my_dir',
                           'Directory where checkpoints and event logs are written to.')
tf.app.flags.DEFINE_integer()
tf.app.flags.DEFINE_float()
tf.app.flags.DEFINE_boolean()

FLAGS = tf.app.flags.FLAGS
```

---
### 1. Train

* pseudo code (flow를 나타낸다.)

```python
with tf.Graph().as_default():
  # Build the model

  # Create global step
  # Calculate the learning rate schedule
  # Decay the learning rate exponentially based on the number of steps

  # Create an optimizer
  # Minimize optimizer

  # Set up the Saver

  # Start session
  with tf.Session() as sess:
    # start the queue runners

    # Create a summary writer
    # Build the summary operation

    for step in range(FLAGS.max_step):
      _, loss = sess.run([opt_op, model.loss])
      # Print loss and save the model checkpoints periodically
```

#### Build the model

```python
import model

model = model.MyModel(mode="train")
model.build()
```

모델 코드 (`model.py`) 작성은 다음 2. Model에서 다룸

#### Create global step

```python
global_step = tf.train.get_or_create_global_step()
```

`tf.train`을 이용하여 `global_step`을 생성한다.
`global_step`은 현재 학습상태가 전체적으로 봤을때 몇 번째 step인지 저장하고 있다.
이를 이용하여 learning rate decay schedule에 이용한다.

#### Calculate the learning rate schedule

```python
num_batch_per_epoch = (FLAGS.num_examples / FLAGS.batch_size)
decay_steps = int(num_batch_per_epoch * FLAGS.num_epochs_per_decay)
```

FLAGS(hyper-parameters들의 argument)를 이용하여 learning rate를 몇 스텝에 한번씩
decay할지 결정함.

#### Decay the learning rate exponentially based on the number of steps

```python
learning_rate = tf.train.exponential_decay(
      FLAGS.initial_learning_rate,
      global_step,
      decay_steps=decay_steps,
      decay_rate=FLAGS.learning_rate_decay_factor,
      staircase=True)
```

[`tf.train`](https://www.tensorflow.org/api_guides/python/train) 모듈에는 train에 필요한
많은 api들이 제공된다.
다양한 종류의 learning rate decay 함수 부터, 바로 다음에 나올 Optimizer, batch 함수,
file 입출력, model save 관련 함수들이 있다.
* `tf.train.exponential_decay`
* `tf.train.inverse_time_decay`
* `tf.train.natural_exp_decay`
* `tf.train.piecewise_constant`
* `tf.train.polynomial_decay`

#### Create an optimizer and minimize optimizer

```python
opt = tf.train.AdamOptimizer(learning_rate)
opt_op = opt.minimize(model.loss,
                      global_step=global_step)
```

[`tf.train.Optimizer`](https://www.tensorflow.org/api_docs/python/tf/train/Optimizer) 모듈에는
여러가지 optimizer 함수들이 제공된다.
* `tf.train.Optimizer`
* `tf.train.GradientDescentOptimizer`
* `tf.train.AdadeltaOptimizer`
* `tf.train.AdagradOptimizer`
* `tf.train.AdagradDAOptimizer`
* `tf.train.MomentumOptimizer`
* `tf.train.AdamOptimizer`
* `tf.train.FtrlOptimizer`
* `tf.train.ProximalGradientDescentOptimizer`
* `tf.train.ProximalAdagradOptimizer`
* `tf.train.RMSPropOptimizer`

optimizer는 기본적으로 `learning_rate` argument가 필요하다.

#### Set up the Saver

```python
# Create a saver
saver = tf.train.Saver(max_to_keep=1000)

# Save the model
with tf.Session() as sess:
  for step in xrange(10000):
	sess.run(opt_op)
	  if step % 100 == 0:
		saver.save(sess, 'my-model', global_step=step)
```

[`tf.train.Saver`](https://www.tensorflow.org/api_docs/python/tf/train/Saver) 모델을 저장 또는
불러올때 사용된다.

#### Start session

```python
# Build an initialization operation to run below
init = tf.global_variables_initializer()

# Start running operations on the Graph
sess = tf.Session()
sess.run(init)
```

위와 같이 session을 생성할 수 있다.
TensorFlow는 `Session`개념이 굉장히 중요하고 이것이 일반적인 프로그래밍과는 다른 차별점을 준다.
사실 이 부분 때문에 초보자들이 프로그래밍과 디버깅하기 어려운 것 같다.
`sess = tf.Session ()`이 생성되기 전까지 위의 프로그래밍은 사실 세팅만 한 것에 불과하다.
저 위의 코드들은 마치 파이프 라인만 연결해 놓은 상태인 것이다.
session을 열고 실행할 때 물이 흐르고 실제로 모든 계산이 시작된다.
위의 코드는 간단히 `with`문으로 구현할 수 있다.

```python
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
```

#### start the queue runners
```phthon
tf.train.start_queue_runners(sess=sess)
```
[`Threading and Queues`](https://www.tensorflow.org/programmers_guide/threading_and_queues)
페이지를 참조

#### Create a summary writer and build the summary operation

```python
# Create a summary writer
summary_writer = tf.summary.FileWriter(FLAGS.train_dir, sess.graph)

# Build the summary operation
summary_op = tf.summary.merge_all()

# Write summary events periodically
with tf.Session() as sess:
  for step in xrange(10000):
    sess.run(opt_op)
    if step % 200 == 0:
      summayr_str = sess.run(summary_op)
      summary_writer.add_summary(summary_str, step)
```

[`tf.summary`](https://www.tensorflow.org/api_docs/python/tf/summary) 페이지 참조.
`tf.summary.FileWriter`를 생성하고, summary operation `summary_op`으로
학습관련 summary들을 다 모은다.
그리고 `Session`에서 원하는 주기마다 `summary_writer`에 summary 저장.

#### Compute loss

```python
for step in range(FLAGS.max_step):
  _, loss = sess.run([opt_op, model.loss])
  # Print loss and save the model checkpoints periodically
  if step % 10 == 0:
    print('step: %d  loss: %f' % (step, loss))

  if step % FLAGS.save_steps == 0:
    checkpoint_path = os.path.join(FLAGS.train_dir, 'model.ckpt')
    saver.save(sess, checkpoint_path, global_step=step)
```

---
### 2. Model

* 주로 모델을 구현할 때 `tf.contrib.slim` high-level wrapper를 사용한다.
* 모델을 하나의 클래스로 만들고 보통 다섯가지 기본 module들을 만든다.
  * `__init__`
  * `build_inputs`
  * `build_network`
  * `build_model` (for loss computation)
  * `build`

####  `__init__`

* 초기화에 필요한 변수들을 argument 형태로 넣는다.
  * `mode in ['train', 'val', 'inference']`
  * `FLAGS`형태의 global variable도 모델안에서 쓸 수 있게 `self` 변수로 저장한다.
  * 반드시 필요한 변수인 `loss`, `global_step`등도 `self`로 가지고 있는다.
  

#### `build_inputs`

* train, validation에서는 tensor형태로 변수를 선언하고
  inference 모드에서는 외부에서 받을 수 있게 `tf.placeholder`형태로 선언한다.
  

#### `build_network`

* 열심히 만든다.
* 구조를 고려하여 `variable_scope`로 적절히 묶어주자.
* 레이어마다 공통으로 쓰이는 hyper parameter들은 `arg_scope`로 묶어서 사용하자.
* 맨 마지막 레이어는 `loss`를 계산하기 위한 `logits`형태로 남겨두자 (**NO activation**)


#### `build_model`

* train, validation, inference 상황에 맞는 `loss`, `eval` 또는 `probability`값을 선언하자.


#### `build`

* 모든 build 단계를 합친다.